# Deep Q-Network (DQN)

The original paper published on Nature can be found [here](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf). And there are two exercellent learning materials for it can be found from [@Hvass-Labs](https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/16_Reinforcement_Learning.ipynb) and [@dennybritz](https://github.com/ZhiruiFeng/reinforcement-learning/tree/master/DQN). 

As metioned by its creator, DeepMind, its meaning lies on:
> This work represents the first demonstration of a general-purpose agent that is able to continually adapt its behavior without any human intervention, a major technical step forward in the quest for general AI.

<img src="./resource/images/dqn-1.png" alt="Drawing" style="width: 800px;"/>

The above figure is borrowed from a [Nature New&View article](https://www.nature.com/articles/518486a.epdf?referrer_access_token=rJi2LNPaO_wh7LCXE8J0gNRgN0jAjWel9jnR3ZoTv0M4DtkukdMkIcR-UVrz0pNp311MkppKL7NysMmwcju-Md7bwkauG8hqmn4c75o_6pA%3D&tracking_referrer=http%3A%2F%2Fwww.nature.com%2Fnews%2Fnewsandviews) which talks about DQN.

Till now, it has become the scaffold of Deep Reinforcement Learning, and offers the core architecture for further improvements, which makes it's really worthwile for everyone who want to work on RL to have a deep and thoroughly look at this model.

And within project **RL-League**, you will find nearlly all other complex models are extensions of this one.

## What is DQN?

We can see DQN as Q-learning algorithm with a Neural Network plays the role of action-value function approximator, which enables the agent learn to choose actions from high-dimentional raw-image input. In Q-learning, $Q^*(s,a)$ represents the accumulated future reward, $Q^*$, if in state $s$ the system first performs action $a$, and subsequently follows an optimal policy. For small state space, tabular RL uses a look-up table to update $Q(s,a)$, which fails when the state space become very large. So here in DQN, the system tries to approximate $Q^*$ by using an artificial neural network - a function approximator to do generalization on states.

The loss function for the function approximator - deep q-network is: $$\mathcal{L}_i(w_i) = \mathbb{E}_{s,a,r,s'\thicksim\mathcal{D}_i}[(r+\gamma \max_{a'}Q(s',a';w_i^-)-Q(s,a;w_i))]^2$$

In this equation: 
- $i$ indicate the time; 
- $\mathcal{D}_i$ means the replay buffer for [experience replay](https://medium.com/@Fihezro/experience-replay-deep-reinforcement-learning-ef02b8a1383); 
- $s'$ is the next state after taking action $a$ at state $s$, note that in dynamic environment, the $s'$ may not be determined by $s$ and $a$; 
- $w_i^-$ is the parameter of network for Q-learning target, which will not be updated frequently, but when updated, it will be set as $w_i$, the parameter of Q-Network;
- $\gamma$ is the discount factor;
- $r+\gamma \max_{a'}Q(s',a';w_i^-)$ means the 'ground truth', assume the optimalization of later actions;
- $Q(s,a;w_i)$ is the evaluation result of Q-network, and the $w_i$ is the parameter updated through gradient descent.

## Structure of this notebook

As we mentioned in the introduction of this project, we are going to use a modular view to see these RL models, you can find the reason [here](http://fzruniverse.life/2018/03/24/Modular-Architecture-for-Implementing-RL-Agent/).

<img src="./resource/images/dqn-2.png" alt="Drawing" style="width: 400px;"/>

Modules are implemented in `/agents/modules/`, here we will just invoke these class and combine them into DQN agent. And an encapsulation of DQN agent can be find under `/agents/`. You can use it to play on `/field/`, where those agents solve problems in virtual game simulators.

## Implementation

In [ ]:
import gym
from gym.wrappers import Monitor
import numpy as np
import sys
import os

if "../agents/modules" not in sys.path:
    sys.path.append("../agents/modules")
from accessory import LogQValues, LogReward, 

In [ ]:
class Agent:
    
    def __init__(self, env_name, training, render=False, use_logging=True):
        
        self.env = gym.make(env.name)
        self.num_actions = self.env.action_space.n
        self.training = training
        self.render = render
        self.use_logging = use_logging
        
        # Log-classes are used for restored the logs
        if self.use_logging and self.training:
            self.log_q_values = LogQValues()
            self.log_reward = LogReward()
        else:
            self.log_q_values = None
            self.log_reward = None
        
        self.action_names = self.env.unwrapped.get_action_meanings()
        
        # The probability of choosing a random action should decrease durin the training time.
        self.epsilon_greedy = EpsilonGreedy(start_value=1.0,
                                            end_value=0.1,
                                            num_iterations=1e6,
                                            num_actions=self.num_actions,
                                            epsilon_testing=0.01)
        
        # Setting of hyper-parameters 
        if self.training:
            self.learning_rate_control = LinearControlSignal(start_value=1e-3,
                                                             end_value=1e-5,
                                                             num_iterations=5e6)
            self.loss_limit_control = LinearControlSignal(start_value=0.1,
                                                          end_value=0.015,
                                                          num_iterations=5e6)
            self.max_epochs_control = LinearControlSignal(start_value=5.0,
                                                          end_value=10.0,
                                                          num_iterations=5e6)
            self.replay_fraction = LinearControlSignal(start_value=0.1,
                                                       end_value=1.0,
                                                       num_iterations=5e6)
        else:
            self.learning_rate_control = None
            self.loss_limit_control = None
            self.max_epochs_control = None
            self.replay_fraction = None
        
        # Setting of replay memory
        if self.training:
            self.replay_memory = ReplayMemory(size=200000, num_actions=self.num_actions)
        else:
            self.replay_memory = None
        
        # Create the Neural Network used for estimating Q-values.
        self.model = NeuralNetwork(num_actions=self.num_actions,
                                   replay_memory=self.replay_memory)
        # Log of the rewards
        self.episode_rewards = []
        
    def reset_episode_reward(self):
        """Reset the log of episode-rewards."""
        self.episode_rewards = []
    
    def get_action_name(self, action):
        """Return the name of an action."""
        return self.action_names[action]
    
    def get_lives(self):
        """Get the number of lives the agent has in the game-environment."""
        return self.env.unwrapped.ale.lives()
    
    